In [ ]:
# XOR
import numpy as np
import pandas as pd 
from sklearn.neighbors import KNeighborsRegressor
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from scipy import stats

print(tf.__version__)

# Training Data Set
x_data = np.array([[0,0],
                   [0,1],
                   [1,0],
                   [1,1]], dtype=np.float32)

t_data = np.array([[0],[1],[1],[0]], dtype=np.float32)

model = Sequential()
# model.add(Flatten(input_shape=(2,)))
model.add(Dense(100, activation='sigmoid', input_shape=(2,))) # 이렇게 쓰면 윗줄이랑 합체 가능함 # 일반적으로는 Flatten 안씀
model.add(Dense(6, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=SGD(learning_rate=1e-1), 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

history = model.fit(x_data, 
                    t_data, 
                    epochs=30000, 
                    verbose=0)

predict_val = model.predict(x_data)
result = tf.cast(predict_val >= 0.5, dtype=tf.float32).numpy().ravel()


print(classification_report(t_data.ravel(),result))


In [ ]:
print(history.history.keys())

plt.plot(history.history['accuracy'], color='b')

In [ ]:
#### mnist DNN tensorflow2

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler        # Normalization
from sklearn.model_selection import train_test_split  # train, test 분리
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Raw Data Loading
df = pd.read_csv('/content/drive/My Drive/MachineLearning/data/mnist/train.csv')
display(df.head(), df.shape)

##### 결측치와 이상치는 없음 #####

# Data Split
x_data_train, x_data_test, t_data_train, t_data_test = \
train_test_split(df.drop('label', axis=1, inplace=False), df['label'], test_size=0.3, random_state=0)
# test_size : test set의 비율 (0.3 => 30%)
# random_state : split할 때 랜덤하게 split하게 되는데 이를 일정하게 고정(seed의 개념)

# Min-Max Normalization
scaler = MinMaxScaler()   # scaler = StandardScaler()
scaler.fit(x_data_train)
x_data_train_norm = scaler.transform(x_data_train)
x_data_test_norm = scaler.transform(x_data_test)

del x_data_train
del x_data_test

## TF2.1 구현

model = Sequential()

model.add(Dense(256, 
                activation='relu', 
                kernel_initializer='he_uniform',
                input_shape=(x_data_train_norm.shape[1],))
          )
model.add(Dropout(0.3))
model.add(Dense(128, 
                activation='relu', 
                kernel_initializer='he_uniform'))
model.add(Dropout(0.3))
model.add(Dense(10, 
                activation='softmax', 
                kernel_initializer='he_uniform'))

model.compile(optimizer=Adam(learning_rate=1e-3),
             loss='sparse_categorical_crossentropy',
             metrics=['sparse_categorical_accuracy'])

history = model.fit(x_data_train_norm,
                    t_data_train,
                    epochs=100,
                    verbose=1,
                    batch_size=128,
                    validation_split=0.3)

In [ ]:
result = np.argmax(model.predict(x_data_test_norm), axis=1)
print(classification_report(t_data_test, result))

In [ ]:
import matplotlib.pyplot as plt

print(history.history.keys())
plt.plot(history.history['sparse_categorical_accuracy'], color='r')
plt.plot(history.history['val_sparse_categorical_accuracy'], color='b')
plt.show()